In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# getting url list to feed into extraction
links = pd.DataFrame(pd.read_csv('intern_train.csv'))
links_list = list(links.url)

# Newspaper3k Extraction

In [30]:
from newspaper import fulltext

def get_text(url):
    
    # getting section and location parts (if it exists) from links 
    meta = []
    for part in url.split('/'):
        
        if part.startswith('tp-'):
            meta.append(part[3:])
        
    section = "None"
    location = "None"
    
    if len(meta) == 2:
        section = meta[0]
        location = meta[1]
    elif len(meta) == 1:
        section = meta[0]  
    elif len(meta) == 0:
        pass
    
    # getting the htmls
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    
    # extract the headline
    headline = soup.find_all('h1')[0].text.strip()
    
    # get the text
    text = fulltext(html).replace('\n',' ').replace('  ',' ')
    
    # clean the staff reporter part. usually its the beginning of the news.
    if text.startswith("Staff Reporter"):
        text = text[text.index(":")+2:]
    
    
    return url,section,location,headline,text

In [32]:
get_text(links_list[432])

('http://www.thehindu.com/todays-paper/tp-opinion/UPA-rule/article14768122.ece',
 'opinion',
 'None',
 'UPA rule',
 "There are no doubt lapses in the performance of the UPA government in the three years it has been in power, the failure to control price rise being one of them. However, the government has provided the much-needed stability at the Centre. Its handling of foreign affairs is particularly noteworthy. The Right to Information Act, the National Rural Employment Guarantee Programme, and the civilian nuclear deal with the U.S. are some of its achievements. D.B.N. Murthy, Bangalore The three years of UPA rule have seen some unprecedented social justice measures that seek to involve every section in the nation's development. Though the measures are yet to be implemented in full, a sincere attempt is being made to implement them. Surely the nation cannot become a developed one when large sections remain out of the national mainstream. Let India shine for everyone. Sachin Kurmi, Gh

In [35]:
def create_documents(links_list):
    # this function will take the list of the urls  and extract one by one and save it into a list which will be a 
         # dataframe later on. and saving it as .csv to use it. 
    results = []
    
    for url in links_list:
        label = int(links.label.iloc[links_list.index(url):links_list.index(url)+1])
        
        r = requests.get(url)
        
        
        url,section,location,headline,text=get_text(url)
        
        results.append([url,section,location,headline,text,label])
        
    return results

In [36]:
res = create_documents(links_list)

In [37]:
pd.DataFrame(res).to_csv("extracted_data.csv")